<a href="https://colab.research.google.com/github/awitz23/LinkedIn/blob/main/Prototyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Rectpack
!pip install -q rectpack

#Optimierung Schnittanzahlfeststellung
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     |████████████████████████████████| 9.6 MB 5.4 MB/s 
     |████████████████████████████████| 49 kB 1.9 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [2]:
import pandas as pd
import numpy as np
import re
import rectpack.guillotine as guillotine
import glob
import matplotlib.patches as patches
import rectpack.packer as packer

from heapq import nsmallest

%matplotlib inline
# Visualize
from plotnine import *
import matplotlib.patches as patches
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from seaborn.axisgrid import FacetGrid

from matplotlib import pyplot as plt
from rectpack import newPacker
from os import listdir
from os.path import isfile, join
from pathlib import Path
M = 50000    # big M ist eine sehr große Zahl

#Optimierung Schnittanzahlfeststellung
import json
import pyomo.environ as pyo
import pyomo.kernel as pmo

In [3]:
pd.set_option('max_rows', 20)
pd.set_option('max_colwidth', 4000)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 4000]


In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

from google.colab import drive
drive.mount('/content/drive')

# Pfad zu einem Ordner mit mehreren Dateien
data_path = r"/content/drive/MyDrive/PJS/01_Input/Daten_v1_clean/01_Preprocessed/prep_flyerdata_170_ma_a18.csv"
data_path_team1 = r"/content/drive/MyDrive/PJS/01_Input/Daten_v1_clean/01_Preprocessed/prep_flyerdata_170_ma_a18.csv"

Mounted at /content/drive


In [5]:
packer_list = [guillotine.GuillotineBssfSas, guillotine.GuillotineBssfLas,guillotine.GuillotineBssfSlas,guillotine.GuillotineBssfLlas,guillotine.GuillotineBssfMaxas, guillotine.GuillotineBssfMinas,guillotine.GuillotineBlsfSas,guillotine.GuillotineBlsfLas,guillotine.GuillotineBlsfSlas,guillotine.GuillotineBlsfLlas,guillotine.GuillotineBlsfMaxas,guillotine.GuillotineBlsfMinas,guillotine.GuillotineBafSas,guillotine.GuillotineBafLas,guillotine.GuillotineBafSlas,guillotine.GuillotineBafLlas,guillotine.GuillotineBafMaxas,guillotine.GuillotineBafMinas]
packer_names = ['GuillotineBssfSas', 'GuillotineBssfLas','GuillotineBssfSlas','GuillotineBssfLlas','GuillotineBssfMaxas','GuillotineBssfMinas','GuillotineBlsfSas','GuillotineBlsfLas','GuillotineBlsfSlas',
'GuillotineBlsfLlas','GuillotineBlsfMaxas','GuillotineBlsfMinas','GuillotineBafSas','GuillotineBafLas','GuillotineBafSlas','GuillotineBafLlas','GuillotineBafMaxas','GuillotineBafMinas']

heuristic_list = [packer.SORT_NONE, packer.SORT_AREA, packer.SORT_PERI, packer.SORT_DIFF, packer.SORT_SSIDE, packer.SORT_LSIDE, packer.SORT_RATIO]
heuristic_names = ['SORT_NONE', 'SORT_AREA', 'SORT_PERI', 'SORT_DIFF', 'SORT_SSIDE', 'SORT_LSIDE', 'SORT_RATIO']

#packer_list_test = [guillotine.GuillotineBssfSas, guillotine.GuillotineBssfLas, guillotine.GuillotineBafLas, guillotine.GuillotineBafSlas]
#heuristic_list_test = [packer.SORT_NONE, packer.SORT_AREA, packer.SORT_DIFF]
#packer_names_test = ['GuillotineBssfSas', 'GuillotineBssfLas','GuillotineBafLas', 'GuillotineBafSlas']
#heuristic_names_test = ['SORT_NONE', 'SORT_AREA', 'SORT_DIFF']

packer_list_test = [guillotine.GuillotineBssfSas, guillotine.GuillotineBssfLas]
heuristic_list_test = [packer.SORT_AREA]
packer_names_test = ['GuillotineBssfSas', 'GuillotineBssfLas']
heuristic_names_test = ['SORT_AREA']


# Aufruf

In [30]:
'''***************************************************************************'''
dict_test={"Test": True, "Day":5.0, "Month":1.0, "Visualization":True, "Number_Best_Results":10}

#If "Test" = True --> Only a subset of algorithms and heursitics is used
'''***************************************************************************'''

if dict_test["Test"] == True:
  packer_list = packer_list_test
  packer_names = packer_names_test
  heuristic_list = heuristic_list_test
  heuristic_names = heuristic_names_test

In [31]:
best_rectpack_result, AllData = divConquerPip(data_path, dict_test, packer_list, packer_names, heuristic_list, heuristic_names, data_path_team1)

#opt = pyo.SolverFactory('glpk')
#df = loadData(data_path, dict_test)
#test, len_cuts =  data_to_dict(df)

Flyeralarm:  76252.0
Cuts:  79 bins:  8
Packer_name_1Packing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

NameError: ignored

# Pipeline

In [26]:
def divConquerPip(data_path, dict_day, packer_list, packer_names, heuristic_list, heuristic_names, data_path_team1):
  #Schritt 1: Daten laden, bins, rectangles für Optimierung, df_flyeralarm zu Schnittzahlberechnung von flyeralarm
  bins, rectangles, df_tag = loadData(data_path, dict_day)
  #Schritt 2: Anzahl der Schnitte bei flyeralarm berechnen
  cuts_flyeralarm, bins_flyeralarm = Auswertung_Flyeralarm(data_path_team1, df_tag)
  #print(cuts_flyeralarm, bins_flyeralarm)

  #Schritt 3: Bewertung der besten Varianten anhand der Schnittanzahl und Binanzahl
  print('Cuts: ', cuts_flyeralarm, 'bins: ', bins_flyeralarm)
  iteration = True
  j = 0
  temp_df = packerResult(rectangles, bins, packer_list, packer_names, heuristic_list, heuristic_names, iteration)
  temp_df = boy_assignment(temp_df, temp_df,iteration, j)
  temp_df, df_visualize_1_Iteration = variant_comparison(temp_df, iteration, bins_flyeralarm, cuts_flyeralarm)
  df_data = temp_df
  temp_df = choose_variant(temp_df, dict_day, iteration)

  #Create Dataframe after Firstpacking with x best variants
  iteration = False
  temp_df, df2 = packing_bad_boy(temp_df, bins, packer_list, packer_names, heuristic_list, heuristic_names, iteration)
  df2, df_visualize_2_Iteration = variant_comparison(df2, iteration, bins_flyeralarm, cuts_flyeralarm)
  
  best_rectpack_result = summary(df2, cuts_flyeralarm, bins_flyeralarm)

  return best_rectpack_result, df2


# Auswertung

In [27]:
def simplythebest_1Pack(df):
  i = -1
  best_result=10
  for element in df['score_1Packing']:
    i = i + 1
    if element <= best_result:
      best_result=element
      index_row = i
  df_bestvariant = df.loc[index_row]
  df_best_combination = pd.DataFrame(
    {'Packer_1.Iteration': [df_bestvariant['Packer_name_1Packing']],
     'Sort heuristic_1.Iteration': [df_bestvariant['Heuristic_1Packing']],
     'Number of bins_1.Iteration': [df_bestvariant['Number_of_Bins_1Packing']],
     'Total cuts_1.Iteration': [df_bestvariant['sum_cut_1Packing']],
     'Best score_1.Iteration': [df_bestvariant['score_1Packing']]
    })
  return df_best_combination


In [28]:
def simplythebest(df):
  best_result=10
  for index, row in df.iterrows():
    i=-1
    for element in row['score_2Packing']:
      i=i+1
      if element <= best_result:
        best_result=element
        index_row = index
        element_position = i
  df_bestvariant = df.loc[index_row]
  df_best_combination = pd.DataFrame(
    {'Packer_1.Iteration': [df_bestvariant['Packer_name_1Packing']],
     'Sort heuristic_1.Iteration': [df_bestvariant['Heuristic_1Packing']],
     'Number of bins_1.Iteration': [df_bestvariant['Number_of_Bins_1Packing']],
     'Total cuts_1.Iteration': [df_bestvariant['sum_cut_1Packing']],
     'Best score_1.Iteration': [df_bestvariant['score_1Packing']],
     'Packer_2.Iteration': [df_bestvariant['Packer_name_2Packing'][element_position]],
     'Sort heuristic_2.Iteration': [df_bestvariant['heuristic_2Packing'][element_position]],
     'Number of bins_2.Iteration': [df_bestvariant['Number_of_Bins_2Packing'][element_position]],
     'Total cuts_2.Iteration': [df_bestvariant['sum_cut_2Packing'][element_position]],
     'Best score_2.Iteration': [df_bestvariant['score_2Packing'][element_position]],
     'packing_Layout_1.Iteration': [df_bestvariant['Variant']],
     'packing_Layout_2.Iteration': [df_bestvariant['packer_2Packing'][element_position]]
    })
  return df_best_combination

In [29]:
def summary(df, cuts_flyeralarm, bins_flyeralarm):
  
  best_result_i1 = simplythebest_1Pack(df2)
  best_result_i2 = simplythebest(df2)

  if best_result_i1['Best score_1.Iteration'][0] <= best_result_i2['Best score_2.Iteration'][0]:
    if best_result_i1['Best score_1.Iteration'][0] <= 1.0:
      best_result = 'Rectpack 1.Iteration'
      df_best_result = best_result_i1
    else: 
      best_result = 'Flyeralarm'
  else:
    if best_result_i2['Best score_2.Iteration'] <= 1.0:
      best_result = 'Rectpack 2.Iteration'
      df_best_result = best_result_i2
    else:
      best_result = 'Flyeralarm'


  print('Die von Flyeralarm generierte Lösung benötigt für', bins_flyeralarm, 'Druckbögen im optimalen Fall', cuts_flyeralarm, 'Schnitte.')
  print('Die beste Rectpack Lösung braucht in der ersten Iterations', best_result_i1['Number of bins_1.Iteration'][0], 'Bins und', int(best_result_i1['Total cuts_1.Iteration']), 'Schnitte.')
  print('Dafür wurde', best_result_i1['Packer_1.Iteration'][0], 'als Packing Algorithmus verwendet und', best_result_i1['Sort heuristic_1.Iteration'][0], 'als Sortierheuristik.')
  print('Die beste Rectpack Lösung braucht in der zweiten Iterations', best_result_i2['Number of bins_2.Iteration'][0], 'Bins und', int(best_result_i2['Total cuts_2.Iteration']), 'Schnitte.')
  print('Dafür wurde in der ersten Iteration', best_result_i2['Packer_1.Iteration'][0], 'als Packing Algorithmus verwendet und', best_result_i2['Sort heuristic_1.Iteration'][0], 'als Sortierheuristik und in der zweiten Iteration:', best_result_i2['Packer_2.Iteration'][0], best_result_i2['Sort heuristic_2.Iteration'][0])
  
  print('Das beste Ergebnis erzielte:', best_result)

  return df_best_result

# Daten auslesen

In [20]:
# Daten auslesen und in bins und rectangles aufteilen - Jede Zeile entspricht einem Flyer bzw. einem Behälter 
def loadData(data_path,dict_day):
  df = pd.DataFrame(pd.read_csv(data_path, sep=',', header=0))
  df = round_df(df)
  bins = df.copy()
  df = dailybusiness(df,dict_day)
  if df.empty:
    print('For this day, there are no jobs')
  
  bins['Tuple'] = list(zip(bins.x_sheetsize, bins.y_sheetsize))
  bins = bins['Tuple'].values.tolist()
  
  # Rectangles werden als Tuple gespeichert als vorbereitung für Rectpack
  rectangles = df.copy()
  rectangles['Tuple'] = list(zip(rectangles.width, rectangles.height))
  rectangles = rectangles['Tuple'].values.tolist()
  
  return bins, rectangles, df

# Auslesen eines spezifischen Tags aus den Daten (wird im Dictionary eingestellt)
def dailybusiness(df, dict_day):

  df_daily = df.loc[(df['day'] == dict_day["Day"]) & (df['month'] == dict_day["Month"])]

  return df_daily

# Data preperation

In [17]:
def round_df(df):
  df[['x_left', 'x_right', 'y_bot', 'y_top', 'x_sheetsize',	'y_sheetsize', 'MarBot', 'MarLeft',	'MarRight',	'MarTop']] = df[['x_left', 'x_right', 'y_bot', 'y_top', 'x_sheetsize',	'y_sheetsize', 'MarBot', 'MarLeft',	'MarRight',	'MarTop']].round(decimals = 2)
  return df

In [18]:
#gibt KPI zum Vergleich der verschiedenen Algorithmen aus
def variant_comparison(df, iteration, bins_flyeralarm, cuts_flyeralarm):

  score_list, count_list, bin_list, bin_list2, count_list2, score_list2, Pack1_name_list, Sort1_name_list, Pack2_name_list2, Sort2_name_list2,  Pack1_name_list_list, Sort1_name_list_list, Score1_list_list,  = ([] for i in range(13))
  j = -1
  if iteration == True:
    for index, row in df.iterrows():
      count = row['sum_cut_1Packing']
      bin_number = row['Number_of_Bins_1Packing']
      Pack1_name = row['Packer_name_1Packing']
      Sort1_name = row['Heuristic_1Packing']

      math_count = count/cuts_flyeralarm
      math_bin = bin_number/bins_flyeralarm

      variant_score = (math_count + math_bin)/2
      score_list.append(variant_score)

      count_list.append(count)
      bin_list.append(bin_number)
      Pack1_name_list.append(Pack1_name)
      Sort1_name_list.append(Sort1_name)

    df['score_1Packing'] = score_list

  elif iteration != True:
    for index, row in df.iterrows():
      score_list, count_list, bin_list, Pack1_name_list, Sort1_name_list, Pack2_name_list, Sort2_name_list, Score1_list = ([] for i in range(8))
      j = j + 1
      for i in range(len(df['Packer_name_2Packing'][j])):
        count = row['sum_cut_2Packing'][i]
        bin_number = row['Number_of_Bins_2Packing'][i]
        Pack2_name = row['Packer_name_2Packing'][i] 
        Sort2_name = row['heuristic_2Packing'][i] 

        Pack1_name = row['Packer_name_1Packing']
        Sort1_name = row['Heuristic_1Packing']
        Score1 = row ['score_1Packing']

        math_count = count/cuts_flyeralarm
        math_bin = bin_number/bins_flyeralarm   

        variant_score = (math_count + math_bin)/2
        score_list.append(variant_score)

        count_list.append(count)
        bin_list.append(bin_number)
        Pack2_name_list.append(Pack2_name)
        Sort2_name_list.append(Sort2_name)

        Pack1_name_list = [Pack1_name] * len(df['Packer_name_2Packing'][j])
        Sort1_name_list = [Sort1_name] * len(df['Packer_name_2Packing'][j])
        Score1_list = [Score1] * len(df['Packer_name_2Packing'][j])

      Pack1_name_list_list.append(Pack1_name_list)
      Sort1_name_list_list.append(Sort1_name_list)
      Score1_list_list.append(Score1_list)

      score_list2.append(score_list)
      count_list2.append(count_list)
      bin_list2.append(bin_list)
      Pack2_name_list2.append(Pack2_name_list)
      Sort2_name_list2.append(Sort2_name_list)

      Pack2_name_list = Pack2_name_list2
      Sort2_name_list = Sort2_name_list2
      score_list = score_list2
      count_list = count_list2
      
      bin_list = bin_list2

    df['score_2Packing'] = score_list

  if iteration == True:
    df_visualize = pd.DataFrame(
      {'Cut_Count': count_list,
      'Bin_Number': bin_list,
      'Packer_name_1': Pack1_name_list,
      'Sort_name_1': Sort1_name_list,
      'Score_1' : score_list
      })
    conditions=[(df_visualize['Packer_name_1'].str.contains('Bssf')),(df_visualize['Packer_name_1'].str.contains('Baf')),(df_visualize['Packer_name_1'].str.contains('Blsf'))]
    choices=['Bssf','Baf','Blsf']
    df_visualize['Rect_choice']=np.select(conditions,choices)
    conditions2=[(df_visualize['Packer_name_1'].str.contains('Sas')),(df_visualize['Packer_name_1'].str.contains('Las')),(df_visualize['Packer_name_1'].str.contains('Slas')),(df_visualize['Packer_name_1'].str.contains('Llas')),(df_visualize['Packer_name_1'].str.contains('Maxas')),(df_visualize['Packer_name_1'].str.contains('Minas'))]
    choices2=['Sas','Las','Slas','Llas','Maxas','Minas']
    df_visualize['AxisSplit_choice']=np.select(conditions2,choices2)
    
  elif iteration != True:
    df_visualize = pd.DataFrame(
    {'Cut_Count': count_list,
     'Bin_Number': bin_list,
     'Packer_name_1': Pack1_name_list_list,
     'Sort_name_1': Sort1_name_list_list,
     'Packer_name_2': Pack2_name_list,
     'Sort_name_2' : Sort2_name_list,
     'Score1' : Score1_list_list,
     'Score2' : score_list
     })
    
  return df, df_visualize

In [19]:
#wählt die besten X Vairanten anhand des ermittelten Scores aus (siehe oben), X kann im Dict definiert werden 
def choose_variant(df, dict_day, iteration):
  if iteration == True:    
    df = df.sort_values(by=['score_1Packing'])
    # Überprüfung ob df genügend Varianten hat um die Top-X auszuwählen (dient zum Testen auf wenig Daten)
    if len(df) >= dict_day['Number_Best_Results']:
      df = df[:dict_day['Number_Best_Results']]
    df = df.reset_index(drop=True)
    print("First iter choose variant worked")

  return df

# Auswertung der Daten von Flyeralarm

In [16]:
def Auswertung_Flyeralarm(data_path_team1, df):
  # Ließt alle Schnittbögen (aus den präparierten Daten) anhand der JobID die an einen definierten Tag erstellt wurden um die Summe der benötigten Schnitte und Bins auszurechnen. 
  # Ausgabe ist die Anzahl der benötigten Schnitte und Bins für den vorbestimmten Tag.

  number_rectangles = len(df)
  width = df['width'].sum()
  height = df['height'].sum()
  sum_area = width + height
  print('Flyeralarm: ', sum_area)
  df_complete = df.copy()
  df = df.drop_duplicates(subset='JobID', keep="first")
  count_cuts_day = df['count_cut'].sum()
 
  count_bins_day = len(df)

  return count_cuts_day, count_bins_day

# Rectpack Funktionen

In [11]:
# Berechnet für jede Kombination von Packing- und Sort-Algorithmen die Anzahl von benötigten Behältern(Bins) und speichert die Ergebnisse in ein DataFrame.
def packerResult(rectangles, bins, packer_list, packer_names, heuristic_list, heuristic_names, iteration):
  rect_num, pack_heu_list, result_pack_name, result_heu_name, result, result_rectangles = ([] for i in range(6))

  i = 0

  for pack in packer_list:
    name = packer_names[i]
    i = i + 1
    j = 0  
    for heuristic in heuristic_list:
      heu_name = heuristic_names[j]
      j = j + 1
      packer, max_value, rect_num = packerCreation(pack, heuristic, rectangles, bins)
      result_pack_name.append(name)
      result_heu_name.append(heu_name)
      result.append(max_value) 
      result_rectangles.append(rect_num)
      pack_heu_list.append(packer)

      if iteration == True:
        df = pd.DataFrame(list(zip(result_pack_name, result_heu_name, result, pack_heu_list, result_rectangles)),
               columns =['Packer_name_1Packing', 'Heuristic_1Packing', 'Number_of_Bins_1Packing', 'packer_1Packing', 'rectangles'])    
      else:
        df = pd.DataFrame(list(zip(result_pack_name, result_heu_name, result, pack_heu_list, result_rectangles)),
               columns =['Packer_name_2Packing', 'Heuristic_2Packing', 'Number_of_Bins_2Packing', 'packer_2Packing', 'rectangles'])
  return df

# Packt alle Rechtecke in die Behälter.
def packerCreation(pack, heuristic, rectangles, bins):
  packer = newPacker(pack_algo = pack, rotation = 1, sort_algo = heuristic)
  for r in rectangles:
    packer.add_rect (*r )
  for b in bins :
    packer.add_bin(*b)  
  packer.pack()
  rect_num = packer.rect_list()
  rect_num = pd.DataFrame(rect_num)
  max_value = rect_num[0].max()
  max_value = max_value + 1 #rect_list startet bei 0 nicht 1
  return packer, max_value, rect_num

def boy_assignment(df, df_gesamt, iteration, j):
  #In dieser Methode werden 4 Schritte ausgeführt:
  #Schritt 1: Anhand der extrahierten Daten wird die Schnittanzahl berechnet.
  #Schritt 2: Die Daten werden in eine Form gebracht so dass der Packingalgorithmus angewandt werden kann.
  #Schritt 3: Es wird für jedes Bin überprüft ob dieses (nahezu) optimal gepackt ist.
  #Schritt 4: Das Daten werden mit weiteren Informationen angereichert (Packer_Name, verwendete Heuristik, Zahl der benötigten Bins etc.)

  cut_list, sum_cutlist_before, sum_cutlist_after, tuple_list= ([] for i in range(4))
  k = 1
  for index, row in df.iterrows():
    print(row)
    sum_cut = cut_count_amounter(row, iteration)
    print(k, '- Iteration durchgelaufen')
    k = k + 1
    rectangle = 'rectangles'
    df_temp = back_to_the_origin(row, rectangle)
    df_temp = check_badboy(df_temp, rectangle)
    cut_list.append(sum_cut)
    if iteration != True:
      sum_cut_after = sum_cut + df_gesamt['sum_gb_1Packing'][j]
      sum_cut_before = df_gesamt['sum_cut_1Packing']
      sum_cutlist_after.append(sum_cut_after)
      sum_cutlist_before.append(sum_cut_before)
    tuple_of_df = list(df_temp.itertuples(index=False, name=None))
    tuple_list.append(tuple_of_df)

  df2 = pd.Series(tuple_list).to_frame('Variant')
  if iteration == True:
    df2[['packer_1Packing', 'Packer_name_1Packing', 'Heuristic_1Packing', 'Number_of_Bins_1Packing']] = df[['packer_1Packing', 'Packer_name_1Packing', 'Heuristic_1Packing', 'Number_of_Bins_1Packing']]
    df2['sum_cut_1Packing'] = cut_list
  else:
    df2[['packer_1Packing', 'Packer_name_1Packing', 'Heuristic_1Packing', 'Number_of_Bins_1Packing']] = df_gesamt[['packer_1Packing', 'Packer_name_1Packing', 'Heuristic_1Packing', 'Number_of_Bins_1Packing']]
    df2['sum_cut_1Packing'] = cut_list
    df2['sum_cut_2Packing'] = sum_cutlist_after
    df2[['packer_2Packing', 'Packer_name_2Packing', 'heuristic_2Packing', 'Number_of_Bins_2Packing']] = df[['packer_2Packing', 'Packer_name_2Packing', 'Heuristic_2Packing', 'Number_of_Bins_2Packing']]
    print('Variant worked')
   
  return df2

In [12]:
def cut_count_amounter(df, iteration):
  #Ermittelt die Anzahl der Schnitte die benötigt werden um die Rectangles (Flyer) aus einem Bin mit Guillotinen-Schnitt herauszutrennen. 
  count_cut_list = []
  sum_obj_value = 0

  if iteration == True:
    Number_of_Bins = df['Number_of_Bins_1Packing']
  else:
    Number_of_Bins = df['Number_of_Bins_2Packing']
  
  df['rectangles']['count_cut'] = 'nan'
  u = 1
  for y in range(Number_of_Bins):
    data = {
      'I': {None: []},
      'KI': {None: []},
      'r': {},
      'h': {},
      'a': {},
      'e': {},
      'Kr': {},
      'Kh': {},
      'Ka': {},
      'Ke': {}
  }
    bin = df['rectangles'].loc[df['rectangles'][0] == y]
    data2 ,Kr_list, Kh_list, Ka_list, Ke_list = create_dict_rectpack(bin, data)
    data3, len_cuts = possible_cuts(data2,Kr_list, Kh_list, Ka_list, Ke_list)
    number_poss_cuts = len(data3['r'].keys()) - 1
    
    data2 = {None: {}}
    data2[None] = data3 
    data = data2

    #print(data)
    print('Wir sind beim ', u, '-Bin')
    u = u + 1
    test, opt_value = Opt_cut(data, number_poss_cuts)
    df['rectangles']['count_cut'] = opt_value
    print(opt_value)
    sum_obj_value =  sum_obj_value + opt_value
    #print(sum_obj_value)

  sum_cut = sum_obj_value

  return sum_cut

In [13]:
def back_to_the_origin(df, subgroup):
  # Daten werden nach der Anwendung des Packing-Algorithmus wieder in die ursprüngliche Form zurück geführt.
  rectangle, rec_in_bin, bin_id_list, bin_list = ([] for i in range(4))

  i, Margin = 0, 0
  for index, row in df[subgroup].iterrows():
    x_right = row[1] + row[3]
    y_top = row[2] + row[4]
    cut, bin_id, height, width = row['count_cut'], row[0], row[4], row[3]
    rectangle = [row[1], row[2], x_right, y_top, width, height, Margin, Margin, Margin, Margin, cut, bin_id]
    if row[0] == i:
      rec_in_bin.append(rectangle)
    elif row[0] != i:
      bin_list.append(rec_in_bin)
      rec_in_bin = []
      rec_in_bin.append(rectangle)
      bin_id_list.append(i)
      i = i + 1
  
  bin_list.append(rec_in_bin)
  bin_id_list.append(i)

  df2 = df[subgroup]
  df2 = df2[[0, 'count_cut']]
  tuples = list(zip(bin_id_list, bin_list))
  df2 = df2.drop_duplicates(subset=[0, 'count_cut'], keep='first')
  df2['rectangles'] = tuples
  df2 = df2.rename(columns={"0": "bin_id"})

  return df2

In [14]:
#In dieser Methode werden mehrere Schritte ausgeführt:
#Schritt 1: Bestimmung der Schnittanzahl der nahezu optimal gepackten Bin (gb) und der nicht optimal gepackten Bins (bb).
#Schritt 2: Herausfiltern der Rectangles (Flyer) aus nicht optimal gepackten Bins zum erneuten Packen
#Schritt 3: Packen der Rectangles
#Schritt 4: Aufruf der Methode "boy_assignment" (Schnittanzahl berechnen, Daten nach dem Packen in die ursrpüngliche Form bringen etc.)

def packing_bad_boy(df, bins, packer_list, packer_names, heuristic_list, heuristic_name, iteration):

  scbb_list, tuple_list, tuple_list2, cut_gb, cut_bb, count_gb, packer_list2, packer_name_list2, heuristic_list2, number_bins_list, sum_bins_2list  = ([] for i in range(11))

  for main_variant in df['Variant']:
    print(main_variant)
    cut_number_bb, cut_number_gb, bb_counter, gb_counter = 0, 0, 0, 0
    for all_bins in main_variant:
      print(all_bins)
      cut_number, category = all_bins[1], all_bins[3]
      print(cut_number)
      cut_number = int(cut_number)
      if category == False:
        cut_number_bb = cut_number_bb + cut_number
      elif category == True:
        print()

        cut_number_gb = cut_number_gb + cut_number   
        gb_counter = gb_counter + 1   

    count_gb.append(gb_counter)
    cut_gb.append(cut_number_gb)
    cut_bb.append(cut_number_bb)

  df['sum_gb_1Packing'] = cut_gb
  df['sum_bb_1Packing'] = cut_bb

  j = -1
  for main_variant in df['Variant']:
      j = j + 1
      rectangle_list=[]
      for bin in main_variant:
        if bin[3] == False:
          for rec in bin[2][1]:
            rectangle = [rec[4], rec[5]]
            rectangle_list.append(rectangle)
      #packerResult erstellt ein neues Dataframe. Das Alte wird nicht übergeben.
       
      temp_var_df = packerResult(rectangle_list, bins, packer_list, packer_names, heuristic_list, heuristic_names, iteration)
      temp_var_df = boy_assignment(temp_var_df, df, iteration, j)
      
      sum_bins_2packing = temp_var_df['Number_of_Bins_2Packing'].tolist()
      pack_list = temp_var_df['packer_2Packing'].tolist()
      packer_name_list = temp_var_df['Packer_name_2Packing'].tolist()
      heuristic2 = temp_var_df['heuristic_2Packing'].tolist()
      sum_bins_2list.append(sum_bins_2packing)
      packer_list2.append(pack_list)
      packer_name_list2.append(packer_name_list)
      heuristic_list2.append(heuristic2)
      sum_cut_after = temp_var_df['sum_cut_2Packing'].tolist()
      scbb_list.append(sum_cut_after)
      tuple_list = list(temp_var_df.itertuples(index=False, name=None))
      tuple_list2.append(tuple_list)

  k = -1
  for lists in sum_bins_2list:
    k = k + 1
    y = count_gb[k]
    new_list = [x+y for x in lists]
    number_bins_list.append(new_list)

  # Hier wird ['Variant'] überschrieben im Vergleich zur vorherigen pack iteration  
  dataframe = df
  dataframe['packer_2Packing'] = packer_list2
  dataframe['Packer_name_2Packing'] = packer_name_list2
  dataframe['heuristic_2Packing'] = heuristic_list2
  dataframe['Number_of_Bins_2Packing'] = number_bins_list 
  dataframe['sum_cut_2Packing'] = scbb_list
  dataframe['Variant_second_iteration'] = tuple_list2
    
  return temp_var_df, dataframe

In [15]:
def check_badboy(df, subgroup):
  #Es wird überprüft ob die Bins (nahezu) optimal verpackt wurden. Logik dahinter: Wenn der summierte Flächeninhalt aller Flyer in einer Bin übereinstimmt mit
  #dem Flächeninhalt des Produkts aus der maximalen x- und y-Koordinaten (die Flyer also ein perfektes Rechteck bilden ohne Lücken und durchgezogenen Kanten) wird 
  #die Annahme getroffen dass das Bin nahezu optimal gepackt ist. 
  judgement_list = []

  for bin in df[subgroup]:
    max_breite, max_höhe, total_area  = 0, 0, 0
    for rectangle in bin[1]:
      m_breite, m_höhe, breite, höhe = rectangle[2], rectangle[3], rectangle[4], rectangle[5]
      add_area = breite*höhe
      total_area = total_area + add_area
      if m_breite >= max_breite:
        max_breite = m_breite
      if m_höhe >= max_höhe:
        max_höhe = m_höhe
      area = max_breite*max_höhe
    if area == total_area:
      judgement_list.append(True)
    elif area != total_area:
      judgement_list.append(False)
    
  df['Boy_Category'] = judgement_list

  return df

# 2. Iteration

In [10]:
def packing_bad_boy(df, bins, packer_list, packer_names, heuristic_list, heuristic_name, iteration):

  scbb_list, tuple_list, tuple_list2, cut_gb, cut_bb, count_gb, packer_list2, packer_name_list2, heuristic_list2, number_bins_list, sum_bins_2list  = ([] for i in range(11))

  for main_variant in df['Variant']:
    cut_number_bb, cut_number_gb, bb_counter, gb_counter = 0, 0, 0, 0
    for all_bins in main_variant:
      cut_number, category = all_bins[1], all_bins[3]
      if category == False:
        cut_number_bb = cut_number_bb + cut_number
      elif category == True:
        cut_number_gb = cut_number_gb + cut_number   
        gb_counter = gb_counter + 1   

    count_gb.append(gb_counter)
    cut_gb.append(cut_number_gb)
    cut_bb.append(cut_number_bb)

  df['sum_gb_1Packing'] = cut_gb
  df['sum_bb_1Packing'] = cut_bb

  j = -1
  for main_variant in df['Variant']:
      j = j + 1
      rectangle_list=[]
      for bin in main_variant:
        if bin[3] == False:
          for rec in bin[2][1]:
            rectangle = [rec[4], rec[5]]
            rectangle_list.append(rectangle)
      #packerResult erstellt ein neues Dataframe. Das Alte wird nicht übergeben.
       
      temp_var_df = packerResult(rectangle_list, bins, packer_list, packer_names, heuristic_list, heuristic_names, iteration)
      temp_var_df = boy_assignment(temp_var_df, df, iteration, j)
      
      sum_bins_2packing = temp_var_df['Number_of_Bins_2Packing'].tolist()
      pack_list = temp_var_df['packer_2Packing'].tolist()
      packer_name_list = temp_var_df['Packer_name_2Packing'].tolist()
      heuristic2 = temp_var_df['heuristic_2Packing'].tolist()
      sum_bins_2list.append(sum_bins_2packing)
      packer_list2.append(pack_list)
      packer_name_list2.append(packer_name_list)
      heuristic_list2.append(heuristic2)
      sum_cut_after = temp_var_df['sum_cut_2Packing'].tolist()
      scbb_list.append(sum_cut_after)
      tuple_list = list(temp_var_df.itertuples(index=False, name=None))
      tuple_list2.append(tuple_list)

  k = -1
  for lists in sum_bins_2list:
    k = k + 1
    y = count_gb[k]
    new_list = [x+y for x in lists]
    number_bins_list.append(new_list)

  # Hier wird ['Variant'] überschrieben im Vergleich zur vorherigen pack iteration  
  dataframe = df
  dataframe['packer_2Packing'] = packer_list2
  dataframe['Packer_name_2Packing'] = packer_name_list2
  dataframe['heuristic_2Packing'] = heuristic_list2
  dataframe['Number_of_Bins_2Packing'] = number_bins_list 
  dataframe['sum_cut_2Packing'] = scbb_list
  dataframe['Variant_second_iteration'] = tuple_list2
    
  return temp_var_df, dataframe

# Funktionen Optimierung (Schnittanzahlbestimmung / Pyomo)

In [8]:
def get_abin(df, job_ID):
  abin = df.loc[df['JobID'] == job_ID]
  return abin

def JobID_per_day(df):
  df = df['JobID'].unique()
  job_id_list = df.tolist()
  return df

def create_dict_rectpack(bin, data):
  i = 0
  Kr_list, Kh_list, Ka_list, Ke_list =[], [], [], []
  for index, row in bin.iterrows():
      Kr_list.append(0)
      Kh_list.append(row[2])
      Ka_list.append(row[1])
      Ke_list.append(row[1]+row[3])
      Kr_list.append(0)
      Kh_list.append(row[2]+row[4])
      Ka_list.append(row[1])
      Ke_list.append(row[1]+row[3])
      Kr_list.append(1)
      Kh_list.append(row[1])
      Ka_list.append(row[2])
      Ke_list.append(row[2]+row[4])
      Kr_list.append(1)
      Kh_list.append(row[1]+row[3])
      Ka_list.append(row[2])
      Ke_list.append(row[2]+row[4])

  index_list = []    
  for i in range(len(Kr_list)):
    for j in range(len(Kr_list)):
      if i != j and Kr_list[i] == Kr_list[j] and Kh_list[i] == Kh_list[j] and Ka_list[i] == Ka_list[j] and Ke_list[i] == Ke_list[j]:
        if i not in index_list:
          index_list.append(j)
          
  index_list.sort(reverse=True)
  for ix in index_list:
    Kr_list.pop(ix)
    Kh_list.pop(ix)
    Ka_list.pop(ix)
    Ke_list.pop(ix)

  for i in range(len(Kr_list)):
    data['Kr'][i+1] = Kr_list[i]
    data['Kh'][i+1] = Kh_list[i]
    data['Ka'][i+1] = Ka_list[i]
    data['Ke'][i+1] = Ke_list[i]
  return data, Kr_list, Kh_list, Ka_list, Ke_list

def possible_cuts(data, Kr_list, Kh_list, Ka_list, Ke_list):
  o = 1
  tuple_list = []
  duplicate_list = []
  for i in range(len(Kr_list)):
    for j in range(len(Kr_list)):
      if Kr_list[i] == Kr_list[j] and Kh_list[i] == Kh_list[j]:
        temp_tuple = (Kr_list[i], Kh_list[i], Ka_list[i], Ke_list[j])
        if i == j and temp_tuple not in duplicate_list and temp_tuple not in tuple_list:
          data['r'][o] = Kr_list[i]
          data['h'][o] = Kh_list[i]
          data['a'][o] = Ka_list[i]
          data['e'][o] = Ke_list[i]
          o = o + 1
          duplicate_list.append(temp_tuple)
        elif Ke_list[i] < Ke_list[j]:
          temp_tuple = (Kr_list[i], Kh_list[i], Ka_list[i], Ke_list[j])
          if temp_tuple not in tuple_list and temp_tuple not in duplicate_list:
            data['r'][o] = Kr_list[i]
            data['h'][o] = Kh_list[i]
            data['a'][o] = Ka_list[i]
            data['e'][o] = Ke_list[j]
            tuple_list.append(temp_tuple)
            o = o + 1
  
  len_cuts = len(data['r'].keys()) +1
  len_edges = len(data['Kr'].keys()) +1 

  data['I'] = {None: list(range(1, len_cuts))}
  data['KI'] = {None: list(range(1, len_edges))}

  return data, len_cuts

In [9]:
def data_to_dict(abin):
  job_id_df = JobID_per_day(df)
  sum_obj_value = 0
  for id in job_id_df:
    data = {
      'I': {None: []},
      'KI': {None: []},
      'r': {},
      'h': {},
      'a': {},
      'e': {},
      'Kr': {},
      'Kh': {},
      'Ka': {},
      'Ke': {}
  }
    bin = get_abin(abin)
    data2 ,Kr_list, Kh_list, Ka_list, Ke_list = create_dict(bin, data)
    data3, len_cuts = possible_cuts(data2,Kr_list, Kh_list, Ka_list, Ke_list)
    number_poss_cuts = len(data3['r'].keys()) - 1
    
    data2 = {None: {}}
    data2[None] = data3 
    data = data2

    #print(data)
    test, opt_value = Opt_cut(data, number_poss_cuts)
    sum_obj_value =  sum_obj_value + opt_value
    print(sum_obj_value)
  return data3, len_cuts

# Optimierungsproblem Aufstellung

In [6]:
from numpy.core.shape_base import block
def ObjRule(model):
  return pyo.summation(model.c)

def unique_cuts(model, i):
  #Die Summe der m.c's welche überlappen dürfen max == 1 sein.
  index_list = []
  index_list2 =[]
  temp_len = len(model.I) + 1
  for j in range(1, temp_len):
    # Checkt ob es Schnitte gibt die zusammen Schnitt i bilden
    if model.r[i] == model.r[j] and model.I[i] != model.I[j] and model.h[i] == model.h[j] and (model.a[i] <= model.a[j] and model.e[i] >= model.e[j]):
      temp_tuple = (model.r[j], model.h[j], model.a[j], model.e[j])
      if temp_tuple not in index_list:
        index_list.append(j)
    # Checkt ob Schnitt i ein Teil von einem längeren Schnitt ist
    elif model.r[i] == model.r[j] and model.I[i] != model.I[j] and model.h[i] == model.h[j] and (model.a[i] >= model.a[j] and model.e[i] <= model.e[j]):
      temp_tuple = (model.r[j], model.h[j], model.a[j], model.e[j])
      if temp_tuple not in index_list2:     
        index_list2.append(j)
  if len(index_list) >= 1 and len(index_list2) == 0:
    return model.c[i] + (sum(model.c[index] for index in index_list)/len(index_list)) == 1
  elif len(index_list) >= 1 and len(index_list2) >= 1:
    return model.c[i] + (sum(model.c[index] for index in index_list)/len(index_list)) + (sum(model.c[index] for index in index_list2)) == 1
  elif len(index_list) == 0 and len(index_list2) >= 1:
    return model.c[i] + (sum(model.c[index] for index in index_list2)) == 1
  elif len(index_list) == 0 and len(index_list2) == 0:
    return model.c[i] == 1

def edgeCut(model, i):
  temp_len = len(model.I) + 1
  Cuts_that_cut_edge = []
  for j in range(1, temp_len):
    if model.a[j] <= model.Ka[i] and model.e[j] >= model.Ke[i] and model.h[j] == model.Kh[i] and model.r[j] == model.Kr[i]:
      Cuts_that_cut_edge.append(j)
      model.k[i] = 1
  return sum(model.c[index] for index in Cuts_that_cut_edge) == 1
      
def AllEdges(model, i):
  return sum(model.k[i] for i in range(1, len(model.KI) + 1)) == len(model.KI)

#Erste If-Bedingung prüft ob der Anfang der Kante vor dem Anfang des Schnitts liegt. Dies Trifft zb. zu wenn ein Behälter 
def Guillo_Blocker_Cut(model, i):
  for j in range(1, len(model.KI)):
    # Wenn ein Blocker existiert und dieser Blocker am Rand oder innerhalb des Schnitts liegt aber schon geschnitten wurden, ist der Schnitt möglich.
    if model.Ka[j] < model.h[i] < model.Ke[j] and model.Kr[j] != model.r[i]:
      index_list, index_list2, index_list3, same_direction_cut1, same_direction_cut2  = [], [], [], [], []
      for k in range(1, len(model.I)):
        if model.r[i] == model.r[k] and model.I[i] != model.I[k] and model.h[i] == model.h[k] and (model.a[i] <= model.a[k] and model.e[i] >= model.e[k]):
          temp_tuple = (model.r[k], model.h[k], model.a[k], model.e[k])
          if temp_tuple not in same_direction_cut1:
            same_direction_cut1.append(k)
        elif model.r[i] == model.r[k] and model.I[i] != model.I[k] and model.h[i] == model.h[k] and (model.a[i] >= model.a[k] and model.e[i] <= model.e[k]):
          temp_tuple = (model.r[k], model.h[k], model.a[k], model.e[k])
          if temp_tuple not in same_direction_cut2:
            same_direction_cut2.append(k)
          # Überprüfung reinbauen ob der cut schon in der Liste ist
        if model.r[k] == model.Kr[j] and model.h[k] == model.Kh[j] and model.a[k] <= model.Ka[j] and model.e[k] >= model.Ke[j]:
          print('GOT HERE')
          if model.r[k] == model.Kr[j] and model.Kr[j] != model.r[i] and model.h[k] == model.Kh[j] and (model.a[k] <= model.Ka[j] and model.e[k] >= model.Ke[j]):
            index_list.append(k)
          elif model.r[k] == model.Kr[j] and model.Kr[j] != model.r[i] and model.h[k] == model.Kh[j] and (model.a[k] >= model.Ka[j] and model.e[k] <= model.Ke[j]):
            index_list2.append(k)
          #Geht besser. Falls es funktioniert könnte ich solche Schnitte früher prüfen und nicht jedesmal wenn ich die Kante finde
      return model.c[i]+ sum(model.c[index] for index in same_direction_cut1) + sum(model.c[index] for index in same_direction_cut2) + sum(model.c[index] for index in index_list) + sum(model.c[index] for index in index_list2) == 2
    # Wenn ein Blocker existiert und dieser Blocker am Rand oder innerhalb des Schnitts liegt aber nicht geschnitten wurde, ist der Schnitt nicht möglich.

    else:
      return (0, 1, 1)

In [7]:
def Opt_cut(data, len_cuts):

  m = pyo.AbstractModel()

  m.l = pyo.RangeSet(1,len_cuts)
  m.I = pyo.Set()

  m.KI = pyo.Set()
  m.r = pyo.Param(m.I)
  m.h = pyo.Param(m.I)
  m.a = pyo.Param(m.I)
  m.e = pyo.Param(m.I)
  m.Kr = pyo.Param(m.KI)
  m.Kh = pyo.Param(m.KI)
  m.Ka = pyo.Param(m.KI)
  m.Ke = pyo.Param(m.KI)
  m.c = pyo.Var(m.I, domain=pyo.Binary, initialize = 0)
  m.k = pyo.Param(m.KI, domain=pyo.Binary, mutable = True, initialize = 0)

  m.obj = pyo.Objective(rule=ObjRule)


  m.AllCut = pyo.Constraint(m.KI, rule = AllEdges)
  m.unique = pyo.Constraint(m.l, rule= unique_cuts)
  m.Edge = pyo.Constraint(m.KI, rule = edgeCut)
  m.Cutting = pyo.Constraint(m.l, rule = Guillo_Blocker_Cut)

  i = m.create_instance(data)
  #i.pprint()

  opt = pyo.SolverFactory('glpk')
  opt.solve(i, tee=True)

  #print('Objective Value: ', i.obj())
  obj_value = i.obj()

  #for v in i.component_objects(pyo.Var, active=True):
  #    print("Variable",v)  
  #    for index in v:
  #        print ("   ",index, pyo.value(v[index]))  
  
  return data, obj_value